In [31]:
from database import *

res = await fetch_all_properties()
print(len(res))

369


In [32]:
documents_list = []
document_ids = []
for docu in res:
    if 'description' in docu.keys():
        if docu['description'] is not None:
            documents_list.append(docu['description'])
            document_ids.append(str(docu['_id']))
len(documents_list), len(document_ids)

(263, 263)

In [33]:
import pandas as pd

# create a dataframe with 7 columns
df = pd.DataFrame(columns=['id', 'document', 'query1', 'query2', 'query3', 'query4', 'query5'], dtype=object)
# insert the documents into the first column
df['document'] = documents_list
df['id'] = document_ids
df.head()

,id,document,query1,query2,query3,query4,query5
0,6531349b011af268d2c45974,ertyuiop[,NaN,NaN,NaN,NaN,NaN
1,653181158a587da37ea6e356,qwerty,NaN,NaN,NaN,NaN,NaN
2,6530f9604f267515c92d16ce,Lorem ipsum samir dil olr Lorem ipsum samir di...,NaN,NaN,NaN,NaN,NaN
3,6531ab908a587da37ea6e886,abc,NaN,NaN,NaN,NaN,NaN
4,6532be5ac0eb87aacc73e33b,qwerty,NaN,NaN,NaN,NaN,NaN


## Preprocessing

In [34]:
# max length of document str column
len(df['document'][0])
max_len = [len(doc) for doc in df['document']]
max(max_len)

69420

In [36]:
# split the row where document len is greater than 1024 into multiple rows keeping the same id

def split_document(row, max_len=100):
    """ Split the document in a row if its length exceeds max_len, keeping the same ID. """
    # Get the length of the document
    length = len(row['document'])
    # If the document is longer than max_len, split it
    if length > max_len:
        # Split the text into chunks
        chunks = [row['document'][i:i+max_len] for i in range(0, length, max_len)]
        # Create new rows for each chunk
        return pd.DataFrame({
            'id': [row['id']] * len(chunks),
            'document': chunks,
            'query1': [row['query1']] * len(chunks),
            'query2': [row['query2']] * len(chunks),
            'query3': [row['query3']] * len(chunks),
            'query4': [row['query4']] * len(chunks),
            'query5': [row['query5']] * len(chunks)
        })
    else:
        # Return the original row if no splitting is needed
        return pd.DataFrame([row])

# Apply the function to each row and concatenate the results
new_rows = df.apply(split_document, axis=1)
new_df = pd.concat(new_rows.values.tolist(), ignore_index=True)

new_df.head()

,id,document,query1,query2,query3,query4,query5
0,6531349b011af268d2c45974,ertyuiop[,NaN,NaN,NaN,NaN,NaN
1,653181158a587da37ea6e356,qwerty,NaN,NaN,NaN,NaN,NaN
2,6530f9604f267515c92d16ce,Lorem ipsum samir dil olr Lorem ipsum samir di...,NaN,NaN,NaN,NaN,NaN
3,6530f9604f267515c92d16ce,olr Lorem ipsum samir dil olr Lorem ipsum sami...,NaN,NaN,NaN,NaN,NaN
4,6530f9604f267515c92d16ce,dil olr Lorem ipsum samir dil olr Lorem ipsum ...,NaN,NaN,NaN,NaN,NaN


In [37]:
new_df.to_csv('dataset.csv')